<a href="https://colab.research.google.com/github/juanfer97/juan-rojas-entrega1/blob/main/99_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Importa librerías básicas para manejo de datos y operaciones

import numpy as np
import pandas as pd
import os
import random

In [7]:
# Configura el entorno para Kaggle y descarga el dataset de la competencia
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
# Descomprime el archivo descargado del dataset
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [9]:
#  Carga los archivos CSV de train y test en dataframes de pandas
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [10]:
#  Muestra las columnas del dataframe de entrenamiento
df_train.columns

Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')

In [11]:
# Instala la librería unidecode para normalización de texto
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.9 MB/s eta 0:00:00


### Preprocesado de Datos
En esta sección se realiza la limpieza y transformación de los datos iniciales. Esto incluye la carga de los conjuntos de datos, la identificación y tratamiento de valores faltantes, la normalización de formatos, y la conversión de variables categóricas en representaciones numéricas que puedan ser interpretadas por los algoritmos de machine learning. Este paso es crucial para garantizar que el modelo reciba datos consistentes y de calidad.

Además se lleva a cabo un análisis preliminar de los datos con el fin de comprender mejor su estructura, distribución y relaciones entre variables. Se analizan estadísticas descriptivas, distribuciones, correlaciones y posibles valores atípicos. Esta etapa permite identificar patrones relevantes, así como detectar errores o inconsistencias en los datos que podrían afectar el desempeño del modelo.

In [12]:
 # Normaliza el nombre del programa académico eliminando tildes y poniendo en mayúsculas
import unidecode
def estu_program_normalizado(df):

  def fillna(row):
    if pd.isna(row):
      return row
    row = row.upper().strip()
    row = unidecode.unidecode(row)
    row = ' '.join(row.split())
    return row

  df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(fillna)


In [13]:
#  Muestra los valores únicos de la columna de programa académico
df_train['E_PRGM_ACADEMICO'].unique()

array(['ENFERMERIA', 'DERECHO', 'MERCADEO Y PUBLICIDAD',
       'ADMINISTRACION DE EMPRESAS', 'PSICOLOGIA', 'MEDICINA VETERINARIA',
       'INGENIERIA MECANICA', 'ADMINISTRACIÓN EN SALUD OCUPACIONAL',
       'INGENIERIA INDUSTRIAL', 'ADMINISTRACIÓN FINANCIERA',
       'HOTELERIA Y TURISMO', 'LICENCIATURA EN CIENCIAS SOCIALES',
       'LICENCIATURA EN PEDAGOGIA INFANTIL', 'COMUNICACION SOCIAL',
       'CIENCIA POLITICA',
       'PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL',
       'MAESTRO EN MÚSICA', 'INGENIERIA MECATRONICA', 'TRABAJO SOCIAL',
       'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL',
       'INGENIERIA CIVIL', 'CONTADURIA PÚBLICA',
       'ADMINISTRACION EN SALUD', 'ADMINISTRACIÓN DE EMPRESAS',
       'ESTADISTICA', 'LICENCIATURA EN BIOLOGIA',
       'INGENIERIA AGROINDUSTRIAL', 'ZOOTECNIA',
       'COMUNICACION AUDIOVISUAL',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES-INGLES',
       'COMUNICACION SOCIAL  - PERIODISMO',
       'SEG

In [14]:
# Muestra la cantidad de valores nulos por columna (solo las que tienen nulos)
p = df_train.isna().sum()
p[p!=0]

,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
F_TIENELAVADORA,39773
F_TIENEAUTOMOVIL,43623
E_PAGOMATRICULAPROPIO,6498
F_TIENECOMPUTADOR,38103
F_TIENEINTERNET.1,26629


In [15]:
# Muestra los valores únicos del departamento del programa y define función de normalización
programas = df_train['E_PRGM_DEPARTAMENTO'].unique()
programas

def estu_program_departamento(df):

  department_name = {
      'BOGOTÁ': 'BOGOTA',
      'NARIÑO': 'NARINO'
  }

  df['E_PRGM_DEPARTAMENTO'] = df['E_PRGM_DEPARTAMENTO'].str.strip()
  df['E_PRGM_DEPARTAMENTO'] = df['E_PRGM_DEPARTAMENTO'].replace(department_name)


In [16]:
# Muestra los valores únicos de estrato y define función para imputar nulos con la moda
estrato = df_train['F_ESTRATOVIVIENDA'].unique()
estrato

def fami_estrato(df):

  #Ej: Moda = Estrato 2
  moda = df['F_ESTRATOVIVIENDA'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].apply(fillna)

### Ingeniería de Características
Se introducen nuevas variables o se transforman las existentes para mejorar la capacidad predictiva del modelo. Este proceso puede incluir combinaciones de variables, transformaciones matemáticas, discretización, codificación avanzada, o selección de variables basadas en su importancia. Una buena ingeniería de características puede tener un gran impacto en los resultados del modelo.

In [17]:
#  Muestra la columna de horas semanales de trabajo
df_train['E_HORASSEMANATRABAJA']

,E_HORASSEMANATRABAJA
0,Menos de 10 horas
1,0
2,Más de 30 horas
3,0
4,Entre 21 y 30 horas
...,...
692495,Entre 11 y 20 horas
692496,Más de 30 horas
692497,Menos de 10 horas
692498,Menos de 10 horas


In [18]:
#  Imputa valores nulos en horas semanales de trabajo con la moda
def estu_hora_trabaja(df):

  moda = df['E_HORASSEMANATRABAJA'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].apply(fillna)

In [19]:
#  Imputa valores nulos en valor de matrícula con la moda
def matricula_valor(df):

  moda = df['E_VALORMATRICULAUNIVERSIDAD'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].apply(fillna)

In [20]:
#  Muestra la moda de estrato de vivienda
df_train['F_ESTRATOVIVIENDA'].mode()[0]

'Estrato 2'

In [21]:
# Muestra los valores únicos de si tiene internet en casa

df_train['F_TIENEINTERNET'].unique() #Sale por probabilidad

array(['Si', 'No', nan], dtype=object)

In [22]:
# Muestra los valores únicos de educación del padre
df_train['F_EDUCACIONPADRE'].unique()

array(['Técnica o tecnológica incompleta',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'No sabe',
       'Primaria completa', 'Educación profesional completa',
       'Educación profesional incompleta', 'Primaria incompleta',
       'Postgrado', nan, 'Secundaria (Bachillerato) incompleta',
       'Ninguno', 'No Aplica'], dtype=object)

In [23]:
#  Muestra los valores únicos de educación del padre y define función para imputar nulos con la moda
df_train['F_EDUCACIONPADRE'].unique() #Hay nan

def fami_educacion_padre(df):

  moda = df['F_EDUCACIONPADRE'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].apply(fillna)

In [24]:
#  Muestra los valores únicos de si tiene lavadora y define función para imputar nulos según estrato y probabilidad
df_train['F_TIENELAVADORA'].unique() #Hay nan

def fami_tienelavadora(df):

  percent_si = np.round(df['F_TIENELAVADORA'].value_counts(normalize=True).get('Si', 0), 4)

  def fillna(row):
    estrato = row['F_ESTRATOVIVIENDA']
    if pd.isna(row['F_TIENELAVADORA']):
      if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
          return 'Si'
      elif random.random() <= percent_si:
          return 'Si'
      else:
          return 'No'
    else:
      return row['F_TIENELAVADORA']
  df['F_TIENELAVADORA'] = df.apply(fillna, axis=1)

In [25]:
#  Calcula el porcentaje de 'Si' en la columna de lavadora
percent_si = np.round(df_train['F_TIENELAVADORA'].value_counts(normalize=True).get('Si', 0), 4)
percent_si

np.float64(0.8631)

In [26]:
#  Muestra los valores únicos de si tiene internet y define función para imputar nulos según estrato y probabilidad
df_train['F_TIENEINTERNET'].unique()  # Hay nan

def fami_tieneinternet(df):
    percent_si = np.round(df['F_TIENEINTERNET'].value_counts(normalize=True).get('Si', 0), 4)

    def fillna(row):
        estrato = row['F_ESTRATOVIVIENDA']
        if pd.isna(row['F_TIENEINTERNET']):
            if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
                return 'Si'
            elif random.random() <= percent_si:
                return 'Si'
            else:
                return 'No'
        else:
            return row['F_TIENEINTERNET']

    df['F_TIENEINTERNET'] = df.apply(fillna, axis=1)


In [27]:
#  Muestra los valores únicos de si tiene internet (columna duplicada) y define función para imputar nulos
df_train['F_TIENEINTERNET.1'].unique()  # Hay nan

def fami_tieneinternet_1(df):
    percent_si = np.round(df['F_TIENEINTERNET.1'].value_counts(normalize=True).get('Si', 0), 4)

    def fillna(row):
        estrato = row['F_ESTRATOVIVIENDA']
        if pd.isna(row['F_TIENEINTERNET.1']):
            if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
                return 'Si'
            elif random.random() <= percent_si:
                return 'Si'
            else:
                return 'No'
        else:
            return row['F_TIENEINTERNET.1']

    df['F_TIENEINTERNET.1'] = df.apply(fillna, axis=1)


In [28]:
#  Calcula el porcentaje de 'Si' en la columna de internet
percent_si = np.round(df_train['F_TIENEINTERNET'].value_counts(normalize=True).get('Si', 0), 4)
percent_si

np.float64(0.8898)

In [29]:
#  Muestra los valores únicos de si tiene automóvil y define función para imputar nulos según estrato y probabilidad
df_train['F_TIENEAUTOMOVIL'].unique()  # Hay nan

def fami_tieneautomovil(df):
    percent_si = np.round(df['F_TIENEAUTOMOVIL'].value_counts(normalize=True).get('Si', 0), 4)

    def fillna(row):
        estrato = row['F_ESTRATOVIVIENDA']
        if pd.isna(row['F_TIENEAUTOMOVIL']):
            if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
                return 'Si'
            elif random.random() <= percent_si:
                return 'Si'
            else:
                return 'No'
        else:
            return row['F_TIENEAUTOMOVIL']

    df['F_TIENEAUTOMOVIL'] = df.apply(fillna, axis=1)


In [30]:
#  Calcula el porcentaje de 'Si' en la columna de automóvil
percent_si = np.round(df_train['F_TIENEAUTOMOVIL'].value_counts(normalize=True).get('Si', 0), 4)
percent_si

np.float64(0.3641)

In [31]:
#  Muestra los valores únicos
df_train['E_PRIVADO_LIBERTAD'].unique()

array(['N', 'S'], dtype=object)

In [32]:
#  Muestra los valores únicos de si paga matrícula propia
df_train['E_PAGOMATRICULAPROPIO'].unique()

array(['No', 'Si', nan], dtype=object)

In [33]:
# Celda: Imputa valores nulos en pago de matrícula propia con la moda
def estu_pagamatricula(df):

  moda = df['E_PAGOMATRICULAPROPIO'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['E_PAGOMATRICULAPROPIO'] = df['E_PAGOMATRICULAPROPIO'].apply(fillna)

In [34]:
#  Muestra los valores únicos de si tiene computador y define función para imputar nulos según estrato y probabilidad
df_train['F_TIENECOMPUTADOR'].unique() #Hay nan

def fami_tienecomputador(df):

  percent_si = np.round(df['F_TIENECOMPUTADOR'].value_counts(normalize=True).get('Si', 0), 4)

  def fillna(row):
    estrato = row['F_ESTRATOVIVIENDA']
    if pd.isna(row['F_TIENECOMPUTADOR']):
      if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
          return 'Si'
      elif random.random() <= percent_si:
          return 'Si'
      else:
          return 'No'
    else:
      return row['F_TIENECOMPUTADOR']
  df['F_TIENECOMPUTADOR'] = df.apply(fillna, axis=1)

In [35]:
#  Muestra los valores únicos de la columna FAMI_TIENEINTERNET.1
df_train['F_TIENEINTERNET.1'].unique()

array(['Si', 'No', nan], dtype=object)

In [36]:
#  Muestra los valores únicos de educación de la madre
df_train['F_EDUCACIONMADRE'].unique()

array(['Postgrado', 'Técnica o tecnológica incompleta',
       'Secundaria (Bachillerato) completa', 'Primaria completa',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) incompleta',
       'Educación profesional incompleta',
       'Educación profesional completa', 'Primaria incompleta', nan,
       'Ninguno', 'No Aplica', 'No sabe'], dtype=object)

In [37]:
#  Imputa valores nulos en educación de la madre con la moda
def fami_educacion_madre(df):

  moda = df['F_EDUCACIONMADRE'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].apply(fillna)

In [38]:
#  Muestra los valores únicos de rendimiento global
df_train['RENDIMIENTO_GLOBAL'].unique()

array(['medio-alto', 'bajo', 'alto', 'medio-bajo'], dtype=object)

In [39]:
#  Convierte los valores de rendimiento global a valores numéricos
def rendimiento(df):

  rendimiento_replace = {
      'bajo': 0,
      'medio-bajo': 1,
      'medio-alto': 2,
      'alto': 3
  }

  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].replace(rendimiento_replace)


In [40]:
#  Aplica todas las funciones de limpieza y transformación al dataset
def limpiar_dataset_completo(df, isrendimiento=False):

    if not isrendimiento:
      rendimiento(df)
    estu_program_normalizado(df)
    estu_program_departamento(df)
    fami_estrato(df)
    fami_educacion_padre(df)
    fami_educacion_madre(df)
    fami_tienelavadora(df)
    fami_tieneinternet(df)
    fami_tieneinternet_1(df)
    fami_tieneautomovil(df)
    fami_tienecomputador(df)
    estu_pagamatricula(df)
    estu_hora_trabaja(df)
    matricula_valor(df)

    return df


### Limpieza de los datasets
**Se usan las funciones creadas anteriormente para llevar acaba la tarea de limpieza además de eliminar algunas columnas**

In [41]:
#  Limpia el dataset de entrenamiento
df_train_limpio = limpiar_dataset_completo(df_train)

/tmp/ipython-input-3128835936.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].replace(rendimiento_replace)


In [42]:
print('Columns in df_train:')
print(df_train.columns)

print('\nColumns in df_test:')
print(df_test.columns)

Columns in df_train:
Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')

Columns in df_test:
Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3',
       'INDICADOR_4'],
      dtype='object')


In [43]:
#  Limpia el dataset de test (sin transformar rendimiento)
df_test_limpio = limpiar_dataset_completo(df_test, isrendimiento=True)

In [44]:
#  Elimina la columna FAMI_TIENEINTERNET.1 del test limpio
df_test_limpio.drop(columns=['F_TIENEINTERNET.1'], inplace=True, axis=1)

In [45]:
#  Elimina la columna FAMI_TIENEINTERNET.1 del train limpio
df_train_limpio.drop(columns=['F_TIENEINTERNET.1'], inplace=True, axis=1)

In [46]:
# Convierte los valores numéricos de rendimiento global a texto
def rendimiento_back(df):

  rendimiento_replace = {
      0: 'bajo',
      1: 'medio-bajo',
      2: 'medio-alto',
      3: 'alto'
  }

  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].replace(rendimiento_replace)


### Entrenamiento del Modelo

En esta etapa se construye un modelo de clasificación empleando los datos ya preprocesados. Se selecciona el algoritmo CatBoost por su adecuación al problema y se ajustan sus hiperparámetros para mejorar su rendimiento. Además, es posible aplicar técnicas como validación cruzada o búsquedas automáticas de hiperparámetros, ya sea mediante grid search o métodos aleatorios.

En este punto, el conjunto de datos también se divide en subconjuntos de entrenamiento y validación. Esta separación permite entrenar el modelo con una parte de los datos y evaluarlo con otra, con el objetivo de estimar su comportamiento frente a información nueva. Para ello, se realiza una división estratificada que garantiza que la distribución de clases se mantenga equilibrada en ambos subconjuntos.

In [47]:
# Instala la librería CatBoost
!pip install catboost

# Entrena y evalúa el modelo CatBoost
# Importa las librerías necesarias para el modelo
import catboost as cb
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Crea una copia del dataset limpio
data = df_train_limpio.copy()
# Separa features (X) y variable objetivo (y)
x = data.drop(["RENDIMIENTO_GLOBAL", "ID"], axis=1)
y = data['RENDIMIENTO_GLOBAL']

# Define las columnas categóricas
cat_cols = x.select_dtypes(include=['object']).columns.tolist()

# Divide los datos en entrenamiento y validación (90%-10%)
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.1, random_state=16, stratify=y
)

# Configura el modelo CatBoost con hiperparámetros optimizados
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.07,
    depth=6,
    bootstrap_type='Bernoulli',
    l2_leaf_reg=3,
    thread_count=12,
    auto_class_weights='Balanced',
    subsample=0.70,
    verbose=100,
    task_type='CPU', eval_metric="Accuracy"
)

# Entrena el modelo con los datos
model.fit(x_train, y_train, eval_set=(x_val, y_val), cat_features=cat_cols, use_best_model=True)

# Realiza predicciones sobre el conjunto de validación
predict_y = model.predict(x_val)

# Calcula y muestra la precisión del modelo
acc = accuracy_score(y_val, predict_y)
print(acc)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00
0:	learn: 0.3917183	test: 0.3952550	best: 0.3952550 (0)	total: 4.73s	remaining: 1h 18m 43s
100:	learn: 0.4369456	test: 0.4395428	best: 0.4395428 (100)	total: 6m 59s	remaining: 1h 2m 12s
200:	learn: 0.4420754	test: 0.4445751	best: 0.4445909 (199)	total: 13m 55s	remaining: 55m 22s
300:	learn: 0.4446626	test: 0.4452367	best: 0.4456141 (296)	total: 21m 2s	remaining: 48m 52s
400:	learn: 0.4461585	test: 0.4456246	best: 0.4458082 (370)	total: 28m 5s	remaining: 41m 57s
500:	learn: 0.4473220	test: 0.4453643	best: 0.4458082 (370)	total: 35m 14s	remaining: 35m 5s
600:	learn: 0.4487512	test: 0.4453649	best: 0.4458082 (370)	total: 42m 34s	remaining: 28m 15s
700:	learn: 0.4497236	test: 0.4453726	best: 0.4458082 (370)	total: 49m 41s	remaining: 21m 11s
800:	learn: 0.4508305	test: 0.4462899	best: 0.4464761 (765)	total: 56m 59s	remaining: 14m 9s
900:	learn: 0.4521033	test: 0.4458657	best: 0.4466673 (833)	total: 1h 4m 24s	remai

### Guardado del Modelo
**Guardar el modelo entrenado**

In [50]:
# Guarda los resultados en un archivo CSV

test = df_test_limpio.copy()
y = test['ID']
test.drop(columns=['ID'], inplace=True, axis=1)

predecir = model.predict(test)

submit_df= pd.concat([y, pd.DataFrame(predecir, columns=['RENDIMIENTO_GLOBAL'])], axis=1)

rendimiento_back(submit_df)

submit_df.to_csv('submission.csv', index=False)

In [51]:
# 6. ENVIAR SUBMISSION A LA COMPETENCIA
# =========================================
!kaggle competitions submit \
    -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
    -f submission.csv \
    -m "Mi primera submission desde Colab"

100% 4.08M/4.08M [00:00<00:00, 6.05MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia